In [1]:
import pandas as pd
import numpy as np
import os
import json
import mysql.connector as sql
import streamlit as st
import plotly.express as px

In [2]:
Mydb = sql.connect(host = "localhost",
                   user = "root",
                   password = "Choudhary@24",
                   database = "phonepe")
mycursor = Mydb.cursor(buffered=True)
cursor = Mydb.cursor()

In [3]:
import requests, zipfile, io

url = "https://github.com/PhonePe/pulse/archive/refs/heads/master.zip"
extract_path = "D:/DSprojects/Phonepe/pulse_data"

r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(extract_path)

print("Repo downloaded and extracted successfully.")


Repo downloaded and extracted successfully.


In [4]:
import os
import json
import pandas as pd

# Use raw string or forward slashes
Path_1 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\aggregated\transaction\country\india\state"
agg_trans_list = os.listdir(Path_1)

columns_1 = {
    'State': [], 'Year': [], 'Quarter': [],
    'Transaction_type': [], 'Transaction_count': [], 'Transaction_amount': []
}

for state in agg_trans_list:
    cur_state = os.path.join(Path_1, state)
    if not os.path.isdir(cur_state):
        continue

    agg_year_list = os.listdir(cur_state)

    for year in agg_year_list:
        cur_year = os.path.join(cur_state, year)
        if not os.path.isdir(cur_year):
            continue

        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = os.path.join(cur_year, file)
            try:
                with open(cur_file, 'r') as f:
                    A = json.load(f)

                if 'data' in A and A['data'] and 'transactionData' in A['data']:
                    for i in A['data']['transactionData']:
                        name = i['name']
                        count = i['paymentInstruments'][0]['count']
                        amount = i['paymentInstruments'][0]['amount']

                        columns_1['Transaction_type'].append(name)
                        columns_1['Transaction_count'].append(count)
                        columns_1['Transaction_amount'].append(amount)
                        columns_1['State'].append(state)
                        columns_1['Year'].append(year)
                        columns_1['Quarter'].append(int(file.strip('.json')))
            except Exception as e:
                print(f"Failed to process {cur_file}: {e}")
                continue

df_agg_trans = pd.DataFrame(columns_1)
df_agg_trans.head()


,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05


In [5]:
df_agg_trans['Transaction_amount'] = (df_agg_trans['Transaction_amount'] / 1e6).round(2)
df_agg_trans['Year'] = df_agg_trans['Year'].astype(int)
# Print head
print(df_agg_trans.head())


                       State  Year  Quarter          Transaction_type  \
0  andaman-&-nicobar-islands  2018        1  Recharge & bill payments   
1  andaman-&-nicobar-islands  2018        1     Peer-to-peer payments   
2  andaman-&-nicobar-islands  2018        1         Merchant payments   
3  andaman-&-nicobar-islands  2018        1        Financial Services   
4  andaman-&-nicobar-islands  2018        1                    Others   

   Transaction_count  Transaction_amount  
0               4200                1.85  
1               1871               12.14  
2                298                0.45  
3                 33                0.01  
4                256                0.18  


In [6]:
mycursor.execute("DROP TABLE IF EXISTS aggregated_Trans")

mycursor.execute("""
    CREATE TABLE IF NOT EXISTS aggregated_Trans (
    State VARCHAR(100),    
    Year INT,
    Quarter INT,
    Transaction_type VARCHAR(100),
    Transaction_count BIGINT,
    Transaction_amount DOUBLE
    )
""")

In [7]:
df_agg_trans.dtypes

State                  object
Year                    int64
Quarter                 int64
Transaction_type       object
Transaction_count       int64
Transaction_amount    float64
dtype: object

In [8]:
for i,row in df_agg_trans.iterrows():
    sql = "INSERT INTO aggregated_Trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    Mydb.commit()

In [9]:
mycursor.execute("SELECT * FROM aggregated_Trans LIMIT 10")  # fetch first 10 rows
rows = mycursor.fetchall()

for row in rows:
    print(row)


('andaman-&-nicobar-islands', 2018, 1, 'Recharge & bill payments', 4200, 1.85)
('andaman-&-nicobar-islands', 2018, 1, 'Peer-to-peer payments', 1871, 12.14)
('andaman-&-nicobar-islands', 2018, 1, 'Merchant payments', 298, 0.45)
('andaman-&-nicobar-islands', 2018, 1, 'Financial Services', 33, 0.01)
('andaman-&-nicobar-islands', 2018, 1, 'Others', 256, 0.18)
('andaman-&-nicobar-islands', 2018, 2, 'Recharge & bill payments', 6735, 2.32)
('andaman-&-nicobar-islands', 2018, 2, 'Peer-to-peer payments', 3575, 24.51)
('andaman-&-nicobar-islands', 2018, 2, 'Merchant payments', 603, 1.02)
('andaman-&-nicobar-islands', 2018, 2, 'Financial Services', 59, 0.12)
('andaman-&-nicobar-islands', 2018, 2, 'Others', 368, 0.36)


In [10]:
Path_2 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\aggregated\user\country\india\state"
agg_user_list = os.listdir(Path_2)

columns_2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [], 'Percentage': []}
for state in agg_user_list:
    cur_state = os.path.join(Path_2, state)
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = os.path.join(cur_state, year)
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = os.path.join(cur_year, file)
            with open(cur_file, 'r') as data:
                B = json.load(data)
                try:
                    for i in B["data"]["usersByDevice"]:
                        brand_name = i["brand"]
                        counts = i["count"]
                        percents = i["percentage"]
                        columns_2["Brands"].append(brand_name)
                        columns_2["Count"].append(counts)
                        columns_2["Percentage"].append(percents)
                        columns_2["State"].append(state)
                        columns_2["Year"].append(year)
                        columns_2["Quarter"].append(int(file.strip('.json')))
                except:
                    pass

df_agg_user = pd.DataFrame(columns_2)
df_agg_user


,State,Year,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [11]:
df_agg_user['Percentage'] = (df_agg_user['Percentage'] * 100).round(2)
df_agg_user['Year'] = df_agg_user['Year'].astype(int)
# Print head to confirm
print(df_agg_user.head())

                       State  Year  Quarter   Brands  Count  Percentage
0  andaman-&-nicobar-islands  2018        1   Xiaomi   1665       24.70
1  andaman-&-nicobar-islands  2018        1  Samsung   1445       21.44
2  andaman-&-nicobar-islands  2018        1     Vivo    982       14.57
3  andaman-&-nicobar-islands  2018        1     Oppo    501        7.43
4  andaman-&-nicobar-islands  2018        1  OnePlus    332        4.93


In [12]:
df_agg_user.dtypes

State          object
Year            int64
Quarter         int64
Brands         object
Count           int64
Percentage    float64
dtype: object

In [13]:
df_agg_user.to_csv('Aggregated_User.csv', index=False)

In [14]:
mycursor.execute("DROP TABLE IF EXISTS Aggregate_User")

mycursor.execute("create table Aggregate_User (State varchar(100), Year int, Quarter int, Brands varchar(100), Count int, Percentage double)")
for i,row in df_agg_user.iterrows():
    sql = "INSERT INTO Aggregate_User VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

In [15]:
mycursor.execute("SELECT * FROM Aggregate_User LIMIT 10")  # fetch first 10 rows
rows = mycursor.fetchall()

for row in rows:
    print(row)

('andaman-&-nicobar-islands', 2018, 1, 'Xiaomi', 1665, 24.7)
('andaman-&-nicobar-islands', 2018, 1, 'Samsung', 1445, 21.44)
('andaman-&-nicobar-islands', 2018, 1, 'Vivo', 982, 14.57)
('andaman-&-nicobar-islands', 2018, 1, 'Oppo', 501, 7.43)
('andaman-&-nicobar-islands', 2018, 1, 'OnePlus', 332, 4.93)
('andaman-&-nicobar-islands', 2018, 1, 'Realme', 316, 4.69)
('andaman-&-nicobar-islands', 2018, 1, 'Apple', 229, 3.4)
('andaman-&-nicobar-islands', 2018, 1, 'Motorola', 226, 3.35)
('andaman-&-nicobar-islands', 2018, 1, 'Lenovo', 202, 3.0)
('andaman-&-nicobar-islands', 2018, 1, 'Huawei', 158, 2.34)


In [16]:
path_3 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\map\transaction\hover\country\india\state"
map_Tran_List = os.listdir(path_3)

columns_3 = {'State':[], 'Year':[], 'Quarter':[], 'District':[], 'Count':[], 'Amount':[]}

for state in map_Tran_List:
    cur_state = os.path.join(path_3, state)
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = os.path.join(cur_state, year)
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = os.path.join(cur_year, file)
            with open(cur_file, 'r') as data:
                c = json.load(data)
                
                for i in c["data"]["hoverDataList"]:
                    district = i["name"]
                    count = i["metric"][0]["count"]
                    amount = i["metric"][0]["amount"]
                    columns_3["District"].append(district)
                    columns_3["Count"].append(count)
                    columns_3["Amount"].append(amount)
                    columns_3['State'].append(state)
                    columns_3['Year'].append(year)
                    columns_3['Quarter'].append(int(file.strip('.json')))

Map_Trans = pd.DataFrame(columns_3)
Map_Trans

,State,Year,Quarter,District,Count,Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
20599,west-bengal,2024,4,alipurduar district,15875637,2.099251e+10
20600,west-bengal,2024,4,paschim bardhaman district,56616799,6.968735e+10
20601,west-bengal,2024,4,nadia district,65274337,1.079320e+11
20602,west-bengal,2024,4,birbhum district,36905213,5.778701e+10


In [17]:
Map_Trans.dtypes

State        object
Year         object
Quarter       int64
District     object
Count         int64
Amount      float64
dtype: object

In [18]:
Map_Trans['Amount'] = (Map_Trans['Amount'] / 1e6).round(2)
Map_Trans['Year'] = Map_Trans['Year'].astype(int)
# Print head
print(Map_Trans.head())

                       State  Year  Quarter  \
0  andaman-&-nicobar-islands  2018        1   
1  andaman-&-nicobar-islands  2018        1   
2  andaman-&-nicobar-islands  2018        1   
3  andaman-&-nicobar-islands  2018        2   
4  andaman-&-nicobar-islands  2018        2   

                            District  Count  Amount  
0  north and middle andaman district    442    0.93  
1             south andaman district   5688   12.56  
2                  nicobars district    528    1.14  
3  north and middle andaman district    825    1.32  
4             south andaman district   9395   23.95  


In [19]:
 Map_Trans.to_csv("Map_Trans.csv",index=False)

In [20]:
mycursor.execute("DROP TABLE IF EXISTS Map_Trans")

mycursor.execute("create table Map_Trans (State varchar(100),Year int, Quarter int, District varchar(100), Count int, Amount double)")
for i,row in Map_Trans.iterrows():
    sql = "INSERT INTO Map_Trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

mycursor.execute("SELECT * FROM Map_Trans LIMIT 10")  # fetch first 10 rows
rows = mycursor.fetchall()

for row in rows:
    print(row)

('andaman-&-nicobar-islands', 2018, 1, 'north and middle andaman district', 442, 0.93)
('andaman-&-nicobar-islands', 2018, 1, 'south andaman district', 5688, 12.56)
('andaman-&-nicobar-islands', 2018, 1, 'nicobars district', 528, 1.14)
('andaman-&-nicobar-islands', 2018, 2, 'north and middle andaman district', 825, 1.32)
('andaman-&-nicobar-islands', 2018, 2, 'south andaman district', 9395, 23.95)
('andaman-&-nicobar-islands', 2018, 2, 'nicobars district', 1120, 3.07)
('andaman-&-nicobar-islands', 2018, 3, 'north and middle andaman district', 1283, 4.9)
('andaman-&-nicobar-islands', 2018, 3, 'south andaman district', 13511, 44.27)
('andaman-&-nicobar-islands', 2018, 3, 'nicobars district', 1471, 6.39)
('andaman-&-nicobar-islands', 2018, 4, 'north and middle andaman district', 2137, 7.96)


In [21]:
Path_4 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\map\user\hover\country\india\state"

map_user = os.listdir(Path_4)

Columns_4 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'RegisteredUser': [], 'AppOpens': []}

for state in map_user:
    cur_state = os.path.join(Path_4, state)
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = os.path.join(cur_state, year)
        map_list = os.listdir(cur_year)
        
        for file in map_list:
            cur_file = os.path.join(cur_year, file)
            with open(cur_file, 'r') as data:
                d = json.load(data)
                # Loop through the hoverData dictionary items
                for district, metrics in d["data"]["hoverData"].items():
                    registered_user = metrics["registeredUsers"]
                    app_opens = metrics['appOpens']
                    Columns_4["District"].append(district)
                    Columns_4["RegisteredUser"].append(registered_user)
                    Columns_4["AppOpens"].append(app_opens)
                    Columns_4['State'].append(state)
                    Columns_4['Year'].append(year)
                    Columns_4['Quarter'].append(int(file.strip('.json')))

data_map_User = pd.DataFrame(Columns_4)
data_map_User

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
20603,west-bengal,2024,4,alipurduar district,475688,31842355
20604,west-bengal,2024,4,paschim bardhaman district,1468252,80543469
20605,west-bengal,2024,4,nadia district,1861738,98740305
20606,west-bengal,2024,4,birbhum district,1114220,73465525


In [22]:
data_map_User.dtypes

State             object
Year              object
Quarter            int64
District          object
RegisteredUser     int64
AppOpens           int64
dtype: object

In [23]:
data_map_User.isna().sum()

State             0
Year              0
Quarter           0
District          0
RegisteredUser    0
AppOpens          0
dtype: int64

In [24]:
data_map_User['Year'] = data_map_User['Year'].astype(int)

In [25]:
data_map_User.dtypes

State             object
Year               int64
Quarter            int64
District          object
RegisteredUser     int64
AppOpens           int64
dtype: object

In [26]:
data_map_User.to_csv("Map_User_data.csv",index=False)

In [27]:
mycursor.execute("DROP TABLE IF EXISTS Map_User")

mycursor.execute("create table Map_User (State varchar(100), Year int, Quarter int, District varchar(100), RegisteredUser int, AppOpens int)")
for i,row in data_map_User.iterrows():
    sql = ("INSERT INTO Map_User VALUES (%s,%s,%s,%s,%s,%s)")
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

mycursor.execute("SELECT * FROM Map_User LIMIT 10")  # fetch first 10 rows
rows = mycursor.fetchall()

for row in rows:
    print(row)

('andaman-&-nicobar-islands', 2018, 1, 'north and middle andaman district', 632, 0)
('andaman-&-nicobar-islands', 2018, 1, 'south andaman district', 5846, 0)
('andaman-&-nicobar-islands', 2018, 1, 'nicobars district', 262, 0)
('andaman-&-nicobar-islands', 2018, 2, 'north and middle andaman district', 911, 0)
('andaman-&-nicobar-islands', 2018, 2, 'south andaman district', 8143, 0)
('andaman-&-nicobar-islands', 2018, 2, 'nicobars district', 351, 0)
('andaman-&-nicobar-islands', 2018, 3, 'north and middle andaman district', 1208, 0)
('andaman-&-nicobar-islands', 2018, 3, 'south andaman district', 10474, 0)
('andaman-&-nicobar-islands', 2018, 3, 'nicobars district', 467, 0)
('andaman-&-nicobar-islands', 2018, 4, 'north and middle andaman district', 1553, 0)


In [28]:
Path_5 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\top\transaction\country\india\state/"

top_trans_list = os.listdir(Path_5)

Columns_5 = {
    'State': [],
    'Year': [],
    'Quarter': [],
    'Pincode': [],
    'Transaction_count': [],
    'Transaction_amount': []
}

for state in top_trans_list:
    cur_state = Path_5 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            try:
                with open(cur_file, 'r') as data:
                    A = json.load(data)
                    
                for i in A.get('data', {}).get('pincodes', []):
                    name = i['entityName']
                    count = i['metric']['count']
                    amount = i['metric']['amount']
                    
                    Columns_5['Pincode'].append(name)
                    Columns_5['Transaction_count'].append(count)
                    Columns_5['Transaction_amount'].append(amount)
                    Columns_5['State'].append(state)
                    Columns_5['Year'].append(int(year))
                    Columns_5['Quarter'].append(int(file.strip('.json')))
            except (KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
                continue

# Create DataFrame
df_top_trans = pd.DataFrame(Columns_5)

# Optional: Ensure correct data types
df_top_trans['Transaction_count'] = pd.to_numeric(df_top_trans['Transaction_count'], errors='coerce')
df_top_trans['Transaction_amount'] = pd.to_numeric(df_top_trans['Transaction_amount'], errors='coerce')
df_top_trans['Year'] = pd.to_numeric(df_top_trans['Year'], errors='coerce')
df_top_trans['Quarter'] = pd.to_numeric(df_top_trans['Quarter'], errors='coerce')

# Show DataFrame
df_top_trans


,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
9994,west-bengal,2024,4,711101,6753348,1.049483e+10
9995,west-bengal,2024,4,700059,6662715,8.009476e+09
9996,west-bengal,2024,4,700039,6575693,9.018748e+09
9997,west-bengal,2024,4,734001,6432082,8.996529e+09


In [29]:
df_top_trans.dtypes

State                  object
Year                    int64
Quarter                 int64
Pincode                object
Transaction_count       int64
Transaction_amount    float64
dtype: object

In [30]:
df_top_trans.isna().sum()

State                 0
Year                  0
Quarter               0
Pincode               2
Transaction_count     0
Transaction_amount    0
dtype: int64

In [31]:
missing_rows = df_top_trans[df_top_trans['Pincode'].isna()]
print(missing_rows)

       State  Year  Quarter Pincode  Transaction_count  Transaction_amount
4810  ladakh  2019        4    None               2014        1.009866e+07
4852  ladakh  2020        4    None              13717        3.671160e+07


In [32]:
df_top_trans.at[4810, 'Pincode'] = '194101'
df_top_trans.at[4852, 'Pincode'] = '194101'

In [33]:
df_top_trans['Pincode'].isna().sum()

np.int64(0)

In [34]:
df_top_trans.dtypes

State                  object
Year                    int64
Quarter                 int64
Pincode                object
Transaction_count       int64
Transaction_amount    float64
dtype: object

In [35]:
df_top_trans['Pincode'] = df_top_trans['Pincode'].astype(int)

In [36]:
df_top_trans.dtypes

State                  object
Year                    int64
Quarter                 int64
Pincode                 int64
Transaction_count       int64
Transaction_amount    float64
dtype: object

In [37]:
df_top_trans.to_csv("Top_Trans_data.csv",index=False)

In [38]:
df_top_trans['Transaction_amount'] = (df_top_trans['Transaction_amount'] / 1e6).round(2)

In [39]:
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.77
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.24
2,andaman-&-nicobar-islands,2018,1,744102,969,3.52
3,andaman-&-nicobar-islands,2018,1,744105,685,1.30
4,andaman-&-nicobar-islands,2018,1,744104,340,1.04
...,...,...,...,...,...,...
9994,west-bengal,2024,4,711101,6753348,10494.83
9995,west-bengal,2024,4,700059,6662715,8009.48
9996,west-bengal,2024,4,700039,6575693,9018.75
9997,west-bengal,2024,4,734001,6432082,8996.53


In [40]:
mycursor.execute("DROP TABLE IF EXISTS Top_trans")

mycursor.execute("create table Top_trans (State varchar(100), Year int, Quarter int, Pincode int, Transaction_count int, Transaction_Amount double)")
for i,row in df_top_trans.iterrows():
    sql = ("INSERT INTO Top_trans VALUES (%s,%s,%s,%s,%s,%s)")
    mycursor.execute(sql,tuple(row))
    Mydb.commit()

mycursor.execute("SELECT * FROM Top_trans LIMIT 10")  # fetch first 10 rows
rows = mycursor.fetchall()

for row in rows:
    print(row)

('andaman-&-nicobar-islands', 2018, 1, 744101, 1622, 2.77)
('andaman-&-nicobar-islands', 2018, 1, 744103, 1223, 2.24)
('andaman-&-nicobar-islands', 2018, 1, 744102, 969, 3.52)
('andaman-&-nicobar-islands', 2018, 1, 744105, 685, 1.3)
('andaman-&-nicobar-islands', 2018, 1, 744104, 340, 1.04)
('andaman-&-nicobar-islands', 2018, 1, 744107, 302, 0.56)
('andaman-&-nicobar-islands', 2018, 1, 744301, 283, 0.45)
('andaman-&-nicobar-islands', 2018, 1, 744112, 269, 0.33)
('andaman-&-nicobar-islands', 2018, 1, 744202, 193, 0.24)
('andaman-&-nicobar-islands', 2018, 1, 744302, 168, 0.55)


In [41]:
Path_6 = r"D:\DSprojects\Phonepe\pulse_data\pulse-master\data\top\user\country\india\state/"
user_data = os.listdir(Path_6)
Columns_6 = {'State':[], 'Year':[], 'Quarter':[], 'Pincode':[], 'RegisteredUsers':[]}
for state in user_data:
    cur_state=Path_6 + state + "/"
    Year_data = os.listdir(cur_state)
    for year in Year_data:
        cur_year =cur_state + year + "/"
        Top_file = os.listdir(cur_year)
        for file in Top_file:
            cur_file = cur_year+file
            data = open(cur_file,'r')
            Data = json.load(data)
            for i in Data['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                Columns_6['Pincode'].append(name)
                Columns_6['RegisteredUsers'].append(registeredUsers)
                Columns_6['State'].append(state)
                Columns_6['Year'].append(year)
                Columns_6['Quarter'].append(int(file.strip('.json')))
Top_user_Data = pd.DataFrame(Columns_6)
Top_user_Data

,State,Year,Quarter,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272
...,...,...,...,...,...
9995,west-bengal,2024,4,733134,169596
9996,west-bengal,2024,4,700059,169448
9997,west-bengal,2024,4,711302,146034
9998,west-bengal,2024,4,700150,142151


In [42]:
Top_user_Data.dtypes

State              object
Year               object
Quarter             int64
Pincode            object
RegisteredUsers     int64
dtype: object

In [43]:
Top_user_Data.isna().sum()

State              0
Year               0
Quarter            0
Pincode            0
RegisteredUsers    0
dtype: int64

In [44]:
Top_user_Data['Pincode'] = Top_user_Data['Pincode'].astype(int)
Top_user_Data['Year'] = Top_user_Data['Year'].astype(int)

In [45]:
Top_user_Data.dtypes

State              object
Year                int64
Quarter             int64
Pincode             int64
RegisteredUsers     int64
dtype: object

In [46]:
Top_user_Data.to_csv("Top_User_data.csv",index=False)

In [47]:
mycursor.execute("DROP TABLE IF EXISTS Top_User")
mycursor.execute("create table Top_User (State varchar(100), Year int, Quarter int, Pincode int, RegisteredUsers int)")
for i,row in Top_user_Data.iterrows():
    sql = ("INSERT INTO Top_User VALUES (%s,%s,%s,%s,%s)")
    mycursor.execute(sql,tuple(row))
    Mydb.commit()
mycursor.execute("SELECT * FROM Top_User LIMIT 10")  # fetch first 10 rows
rows = mycursor.fetchall()

for row in rows:
    print(row)

('andaman-&-nicobar-islands', 2018, 1, 744103, 1608)
('andaman-&-nicobar-islands', 2018, 1, 744101, 1108)
('andaman-&-nicobar-islands', 2018, 1, 744105, 1075)
('andaman-&-nicobar-islands', 2018, 1, 744102, 1006)
('andaman-&-nicobar-islands', 2018, 1, 744104, 272)
('andaman-&-nicobar-islands', 2018, 1, 744202, 237)
('andaman-&-nicobar-islands', 2018, 1, 744112, 215)
('andaman-&-nicobar-islands', 2018, 1, 744301, 207)
('andaman-&-nicobar-islands', 2018, 1, 744107, 196)
('andaman-&-nicobar-islands', 2018, 1, 744211, 156)


In [48]:
mycursor.execute("show tables")
mycursor.fetchall()

[('aggregate_user',),
 ('aggregated_trans',),
 ('map_trans',),
 ('map_user',),
 ('top_trans',),
 ('top_user',)]